1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [11]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [6]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [7]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    
from sklearn.preprocessing import StandardScaler

In [8]:
#создадим списки признаков
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [13]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [14]:
#объединим всё в единый пайплайн
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

### LGBM

In [16]:
from lightgbm import LGBMClassifier
classifier_lgbm = Pipeline([
    ('features',feats),
    ('classifier', LGBMClassifier(max_depth=5, random_state = 42)),
])

In [17]:
#обучим наш пайплайн
classifier_lgbm.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('Geography',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))],
                                                          verbose=False)),
                                                ('Gender',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                     

In [21]:
#наши прогнозы для тестовой выборки
preds_lgbm = classifier_lgbm.predict_proba(X_test)[:, 1]


In [20]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [26]:
precision_lgbm, recall_lgbm, thresholds_lgbm = precision_recall_curve(y_test, preds_lgbm)
f_score_lgbm = (2 * precision_lgbm * recall_lgbm) / (precision_lgbm + recall_lgbm)
ix_lgbm = np.argmax(f_score_lgbm)
print('best_tresholds_lgbm= %.3f, f-score_lgbm= %.3f, precision_lgbm= %.3f, recall_lgbm= %.3f' % (thresholds_lgbm[ix_lgbm],
                                                                                                 f_score_lgbm[ix_lgbm],
                                                                                                precision_lgbm[ix_lgbm],
                                                                                                recall_lgbm[ix_lgbm])
     )

best_tresholds_lgbm= 0.309, f-score_lgbm= 0.637, precision_lgbm= 0.584, recall_lgbm= 0.701


In [27]:
cnf_matrix_lgbm = confusion_matrix(y_test, preds_lgbm>thresholds_lgbm[ix_lgbm])
cnf_matrix_lgbm

array([[1737,  254],
       [ 153,  356]], dtype=int64)

### Случайный лес

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
classifier_rf = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [31]:
#обучим наш пайплайн
classifier_rf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('Geography',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))],
                                                          verbose=False)),
                                                ('Gender',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                     

In [33]:
#получим вероятности
preds_rf = classifier_rf.predict_proba(X_test)[:, 1]

In [36]:
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(y_test, preds_rf)
f_score_rf = (2 * precision_rf * recall_rf) / (precision_rf + recall_rf)
ix_rf = np.argmax(f_score_rf)
print('best_tresholds_rf= %.3f, f-score_rf= %.3f, precision_rf= %.3f, recall_rf= %.3f' % (thresholds_rf[ix_rf],
                                                                                                 f_score_rf[ix_rf],
                                                                                                precision_rf[ix_rf],
                                                                                                recall_rf[ix_rf])
     )

best_tresholds_rf= 0.500, f-score_rf= 0.607, precision_rf= 0.667, recall_rf= 0.556


In [37]:
cnf_matrix_rf = confusion_matrix(y_test, preds_rf>thresholds_rf[ix_rf])
cnf_matrix_rf

array([[1912,   79],
       [ 277,  232]], dtype=int64)

### Логистическая регрессия

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
classifier_log_loss = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

In [41]:
classifier_log_loss.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('Geography',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))],
                                                          verbose=False)),
                                                ('Gender',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                     

In [42]:
preds_log_loss = classifier_log_loss.predict_proba(X_test)[:, 1]

In [43]:
precision_log_loss, recall_log_loss, thresholds_log_loss = precision_recall_curve(y_test, preds_log_loss)
f_score_log_loss = (2 * precision_log_loss * recall_log_loss) / (precision_log_loss + recall_log_loss)
ix_log_loss = np.argmax(f_score_log_loss)
print('best_tresholds_log_loss= %.3f, f-score_log_loss= %.3f, precision_log_loss= %.3f, recall_log_loss= %.3f' % (thresholds_log_loss[ix_log_loss],
                                                                                                 f_score_log_loss[ix_log_loss],
                                                                                                precision_log_loss[ix_log_loss],
                                                                                                recall_log_loss[ix_log_loss])
     )

best_tresholds_log_loss= 0.290, f-score_log_loss= 0.510, precision_log_loss= 0.462, recall_log_loss= 0.568


In [44]:
cnf_matrix_log_loss = confusion_matrix(y_test, preds_log_loss>thresholds_log_loss[ix_log_loss])
cnf_matrix_log_loss

array([[1655,  336],
       [ 221,  288]], dtype=int64)

In [49]:
df = pd.DataFrame({'f_score': [f_score_lgbm[ix_lgbm], f_score_rf[ix_rf], f_score_log_loss[ix_log_loss]],
                 'precision': [precision_lgbm[ix_lgbm], precision_rf[ix_rf], precision_log_loss[ix_log_loss]],
                  'recall': [recall_lgbm[ix_lgbm], recall_rf[ix_rf], recall_log_loss[ix_log_loss]]}
                 
                 
                 )

In [51]:
df.index = ['LGBM', 'RandomForest', 'LogLoss']

In [52]:
df.round(3)

,f_score,precision,recall
LGBM,0.638,0.584,0.701
RandomForest,0.607,0.667,0.556
LogLoss,0.510,0.462,0.568


In [ ]:
#Вывод: здесь нужно максимизировать recall, поэтому наилучшая модель- LGBM.
# Но получается, что нам нужно обращать внимание на FP. Если таких будет много,
# то мы впустую потратим деньги на то, чтобы они остались, хотя они и так останутся. Нужно минимизировать FP.


3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

In [65]:
cnf_matrix_lgbm

array([[1737,  254],
       [ 153,  356]], dtype=int64)

In [66]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_rf>thresholds_lgbm[ix_lgbm]).ravel()

In [67]:
expenses = TP + FN
expenses

509

In [68]:
income = 2 * TP
income

638

In [69]:
Rev = income - expenses
Rev

129

In [ ]:
#заработали 129 $

4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [73]:
from sklearn.model_selection import GridSearchCV

params={'classifier__max_depth':[20, 30, 50],
        'classifier__num_leaves':[10, 20, 30],
        'classifier__n_estimators': [100, 200]
        }

In [74]:
grid = GridSearchCV(classifier_lgbm,
                    param_grid=params,
                    cv=5,
                    scoring='recall',
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_depth': 20,
 'classifier__n_estimators': 100,
 'classifier__num_leaves': 30}

In [75]:
#обучим модель с новыми параметрами

classifier_lgbm_search = Pipeline([
    ('features',feats),
    ('classifier', LGBMClassifier(max_depth=20, n_estimators=100, num_leaves=30, random_state = 42)),
])

In [76]:
classifier_lgbm_search.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('Geography',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))],
                                                          verbose=False)),
                                                ('Gender',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                     

In [77]:
preds_lgbm_search = classifier_lgbm_search.predict_proba(X_test)[:, 1]

In [103]:
precision_lgbm_search, recall_lgbm_search, thresholds_lgbm_search = precision_recall_curve(y_test, preds_lgbm_search)
b=1
fb_score_lgbm_search = (1+b**2) * (precision_lgbm_search * recall_lgbm_search) / (b**2 * precision_lgbm_search + recall_lgbm_search)
ix_lgbm_search = np.argmax(fb_score_lgbm_search)
print('best_tresholds_lgbm_search= %.3f, f-score_lgbm_search= %.3f, precision_lgbm_search= %.3f, recall_lgbm_search= %.3f' % (thresholds_lgbm_search[ix_lgbm_search],
                                                                                                 fb_score_lgbm_search[ix_lgbm_search],
                                                                                                precision_lgbm_search[ix_lgbm_search],
                                                                                                recall_lgbm_search[ix_lgbm_search])
     )

best_tresholds_lgbm_search= 0.376, f-score_lgbm_search= 0.646, precision_lgbm_search= 0.640, recall_lgbm_search= 0.652


In [104]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_lgbm_search>thresholds_lgbm_search[ix_lgbm_search]).ravel()
print(TN, FP, FN, TP)

1804 187 178 331


In [105]:
expenses = TP + FN
expenses

509

In [106]:
income = 2 * TP
income

662

In [107]:
Rev = income - expenses
Rev

153

In [ ]:
#заработали 153 $